In [16]:
from langchain_google_genai import GoogleGenerativeAI  
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os 
import pandas as pd

import sqlalchemy
from sqlalchemy import text  # Import text for executing raw SQL
from sqlalchemy import create_engine, inspect


from get_tables_details import GetTabelsDetails


In [17]:
user = 'root'
password = 'mmm_321478910'
host = 'localhost'
db_name = 'FAIRPRICEKE'.lower()
   
        # Create engine without specifying the database to create it if necessary
engine = sqlalchemy.create_engine(
                f'mysql+pymysql://{user}:{password}@{host}/{db_name}'
                )

connection = engine.connect()

In [18]:
# 2. Get unique brand names grouped by category
query_product_name = """
            SELECT Category, GROUP_CONCAT(DISTINCT Brand ORDER BY Brand SEPARATOR ', ') AS Brand
            FROM phoneplacekenya
            GROUP BY Category
            ORDER BY Category;
            """

# Execute the query and load data into a DataFrame
data = pd.read_sql(text(query_product_name), connection)

In [19]:
data

,Category,Brand
0,Apple Accessories,"Apple, iPad, Magic"
1,Apple iPad,"Apple, iPad, Magic"
2,Apple Watch,Apple
3,Buds,"Infinix, OnePlus"
4,Chargers,"Infinix, OnePlus, Samsung"
5,Foldable Phones,"OnePlus, Samsung, Tecno"
6,Galaxy Buds,Samsung
7,Galaxy Tablet,Samsung
8,Galaxy Watch,Samsung
9,Google Pixel,Google


In [20]:
# Format the results in the specified format
result = ""
for category, terms in data.itertuples(index=False):
            terms_list = terms.split(", ")
            result += f"Category {category} incloud:\n"
            for term in terms_list:
                result += f"    - {term}\n"
            result += "\n"  # Add a newline between categories
            
print(result)

Category Apple Accessories incloud:
    - Apple
    - iPad
    - Magic

Category Apple iPad incloud:
    - Apple
    - iPad
    - Magic

Category Apple Watch incloud:
    - Apple

Category Buds incloud:
    - Infinix
    - OnePlus

Category Chargers incloud:
    - Infinix
    - OnePlus
    - Samsung

Category Foldable Phones incloud:
    - OnePlus
    - Samsung
    - Tecno

Category Galaxy Buds incloud:
    - Samsung

Category Galaxy Tablet incloud:
    - Samsung

Category Galaxy Watch incloud:
    - Samsung

Category Google Pixel incloud:
    - Google

Category Infinix incloud:
    - Infinix

Category iPhone incloud:
    - Apple
    - iPhone

Category Itel Phones incloud:
    - Itel

Category Mac Studio incloud:
    - Apple
    - Mac

Category Media Streamers incloud:
    - Apple
    - Xiaomi

Category Nokia Phones incloud:
    - Nokia

Category OnePlus Phones incloud:
    - OnePlus

Category Oppo Phones incloud:
    - Oppo

Category Phone Covers incloud:
    - Galaxy
    - S24
    - 

In [21]:
load_dotenv()
genini_key =  os.getenv("GEMINI_KEY")


llm = GoogleGenerativeAI(
    model='gemini-1.5-flash-latest',
    google_api_key=genini_key)


In [22]:
# Model 1
prompt_model_1 = """
                You are part of a system designed to work as shopping assistant specializing in helping users find the best deals on products from Kenyan e-commerce platforms.
                In this system you are an expert in SQL and database querying.
                Your main task is generate sql queries base on the user questions and the data base schema and tabels are available.
                
                YOU have two tabels in your database :
                     1- jumia Table 
                     2- phoneplacekenya Table
                     
                -------------------------------------------------------------- 
                FOR jumia Table:
                    
                jumia Table Structure:
                
                    {jumia_table_structure}
                    
                Category and Brand in jumia table :
                    {jumia_table_details}
                    
                3 rows example from jumia table :
                timestamp	productName	Brand	jumia_price	jumia_oldPrice	jumia_discount	jumia_rating	jumia_verifiedRatings	jumia_stock	Key_Features	Category
                0	2024-10-03 20:35:39	Samsung Galaxy A05	Samsung	11560.0	12500.0	8.0	4.3 out of 5	(51 verified ratings)	["inStock": true, "stockStatus": "87 items lef...	["Display:\u00a06.7\"\u00a0PLS LCD": true, "Re...	Smart Phones
                1	2024-10-03 20:35:39	Samsung Galaxy A05	Samsung	12930.0	14000.0	8.0	4.7 out of 5	(7 verified ratings)	["inStock": true, "stockStatus": "In stock", "...	["6.7 inches IPS LCD display": true, "Qualcomm...	Smart Phones
                2	2024-10-03 20:35:39	Samsung Fit 3 Smart Watch - Black(1 YR WRTY)	Samsung	10600.0	21200.0	50.0	5 out of 5	(1 verified rating)	["inStock": true, "stockStatus": "9 units left...	["Display: 1.6\" AMOLED Display. 256x402 Resol...	Smart Phones
                    
                    
                
                ----------------------------------------------------------------
                FOR phoneplacekenya Table:
                
                phoneplacekenya Table Structure:
                    {phoneplacekenya_table_structure}
                    
                Category and Brand in phoneplacekenya table :
                    {phoneplacekenya_table_details}
                 
                3 rows example from phoneplacekenya table : 
                timestamp	productName	Brand	PhonePlaceKenya_productLink	price	oldPrice	discount	verifiedRatings	stock	Key_Features	Category
                0	2024-10-12 18:35:57	Nokia C22	Nokia	https://www.phoneplacekenya.com/product/nokia-...	17500.0	18000.0	2.78	0	IN STOCK	["RAM": "4GB", "Storage": "128GB", "Battery": ...]	Nokia Phones
                1	2024-10-12 18:36:06	Nokia C32	Nokia	https://www.phoneplacekenya.com/product/nokia-...	18300.0	18500.0	1.08	0	IN STOCK	["RAM": ", 4GB", "Storage": "64GB,", "Battery"...]	Nokia Phones
                2	2024-10-12 18:36:16	Nokia T21	Nokia	https://www.phoneplacekenya.com/product/nokia-...	28500.0	38500.0	25.97	0	IN STOCK	["RAM": "4GB", "Storage": "64 GB / 128GB", "Ba...]	Nokia Phones
                
                ----------------------------------------------------------------------------------    
                When  generate sql queries base on the user questions, follow these updated guidelines:

                ### 1. **Best Deal Identification**
                - **Scan current prices**, discounts, promotions, and price trends for the products the user asks about.
                - **Identify the platform** with the lowest price, including the exact price, any available discounts, and a **direct link** to the product.
                - Highlight any ongoing promotions, price drops, or special offers relevant to the product in question.

                ### 2. **Product Comparisons**
                - If the user requests a **comparison**, provide a detailed side-by-side breakdown of the prices across multiple e-commerce platforms.
                - Clearly **show the differences in prices, discounts**, and features (e.g., warranty, accessories, etc.).
                - Use historical price data, if available, to predict whether the user should buy now or wait for a price drop.

                ### 3. **Top Deals and Recommendations**
                - For questions regarding the **best deals** (e.g., "top 5 deals"), identify products with the most significant price reductions or best value. Include:
                - Product name, current price, old price, and **percentage discount**.
                - Explain why each deal is valuable

                ### 4. **Platform-Specific Insights**
                - **Analyze platform-specific factors**, such as shipping speed, return policies, and verified customer reviews. Mention these if they impact the user’s decision.
                - If a user asks for the **platform with the widest product range** (e.g., for a brand or category), compare the number  products listed on each platform and provide a clear recommendation.

                ### 5. **Saving Potential and Price Predictions**
                - When asked about potential savings (e.g., "how much can I save on this product?"), calculate the savings in both percentage and absolute terms.
                - If a user asks if a price might drop in the future, analyze historical data and current trends to provide a **prediction with a confidence level** and reasoning.
                
                ### 6. **Brand-Specific Queries**
                - For brand-specific requests, return data only for the requested brand. If no products from that brand are found, state this explicitly.
                - If a user asks about **brand comparisons** (e.g., "Which platform offers the best price for Samsung phones?"), only compare prices for the specific brand without suggesting alternatives.

                ### 7. **Detailed Price History and Forecasting**
                - For questions on **price history**, retrieve historical price data for the past 6 months (or the available range). Present a **clear trend analysis**, and include visual aids like graphs where applicable.
                - If asked about when to buy, analyze the product’s **price trends**, and predict the optimal time to make a purchase, factoring in upcoming sales events or promotions.

                ---

                ### **Important Considerations in Your Output:**
                1. **Price and Discount Information**: Always provide both the current price and the old price, highlighting the percentage discount where applicable. Display this clearly for the user.
                2. **Thorough Data Search**: Perform a deep and thorough search of all available data before providing an answer.
                3. **Direct Links to Products**: Whenever referencing a product, include a direct link to the product on the platform offering the best price or deal.
                4. **Clear, Concise Responses**: Ensure your response is accurate, complete, and addresses the user's question without missing any important details.

                
                ### **Example Questions and Expected Responses:**
                1. **Which platform has the lowest price for [specific product] right now?**
                - The model should return the platform name, current price, any discounts, and a link to the product.

                2. **How does the price of [product] compare across the e-commerce sites in Kenya?**
                - Provide a side-by-side comparison of prices, clearly showing the best offer, including any promotions or discounts.

                3. **What are the 5 (five) best deals in the [category] available right now?**
                - Present a list of the top 5 deals based on price reductions, showing current price, old price, and discount percentage.

                4. **What are the current best-selling phones under 20,000 KES?**
                - Retrieve a list of phones under 20,000 KES from multiple platforms, highlighting the best-selling models based on price drops and customer interest.
                
                5. Are there any ongoing promotions for brand X that I should be aware of?
                -  Should scan current promotions across various e-commerce platforms, identify any active deals or discounts for brand X products, AND present a SUMMARY of the most relevant and valuable offers.

                VERY IMPORTANT : Always RETURN THE QUERIES IN THIS FORMAT ```sql ```
                IMPORTANT: YOU WORK IN MySQL
                IMPORTANT: DON'T MAKE SUM IN ORDER BY, MAKE IT IN SELECT ONLY.
                VERY IMPORTANT: PLEASE MAKE ALL STRING IN Queries INSIDE DOUBLE QUOTE LIKE THIS "" BUT DON'T DO THIS IN COLUMNS NAMES
                IMPORTANT : Output only the SQL query in response.
                VERY IMPORTANT: MySQL does not allow SUM() to be used in the ORDER BY clause the way you have written it. You need to perform the summation in the SELECT clause and use an alias for it in the ORDER BY clause.

      
            
                User question: {question} 


    
"""

 
extraction_prompt_template = PromptTemplate(
    input_variables=[ 'jumia_table_structure' ,'jumia_table_details'  , 'phoneplacekenya_table_details' , 'phoneplacekenya_table_structure','question'],
    template= prompt_model_1
)

# Create the chain
sql_chain = LLMChain(
    llm = llm,
    prompt = extraction_prompt_template
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_18064\744769353.py:125: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sql_chain = LLMChain(


In [33]:
obj = GetTabelsDetails()
    
# Analyze Jumia table
jumia_table_structure  , jumia_table_details= obj.analyze_jumia_table()
    
# Get Phone Place Kenya data
phoneplacekenya_table_structure , phoneplacekenya_table_details = obj.get_phone_place_kenya_data()


In [34]:
query_text = sql_chain.run(
    {'jumia_table_structure' : jumia_table_structure ,
        'jumia_table_details' : jumia_table_details,
        'phoneplacekenya_table_structure' : phoneplacekenya_table_structure,
     'phoneplacekenya_table_details' : phoneplacekenya_table_details,
     'question': 'What are the current best-selling phones under 20,000 KES?'}
)

In [35]:
print(query_text)

```sql
SELECT 
    jumia.productName,
    jumia.jumia_price,
    jumia.jumia_discount,
    phoneplacekenya.productName,
    phoneplacekenya.price,
    phoneplacekenya.discount
FROM
    jumia
JOIN
    phoneplacekenya ON jumia.productName = phoneplacekenya.productName
WHERE
    jumia.jumia_price < 20000 AND phoneplacekenya.price < 20000
ORDER BY
    (jumia.jumia_price * (
        1 - (jumia.jumia_discount / 100)
    )) ASC,
    (phoneplacekenya.price * (
        1 - (phoneplacekenya.discount / 100)
    )) ASC;
```


In [36]:
import re
def extract_query( query_text):
        """
        Extract all SQL queries from the generated query text.
        
        Args:
        - query_text: The text generated by the LLM containing SQL queries.
        
        Returns:
        - A list of extracted SQL queries or None if no matches found.
        """
        pattern = r"```sql\s*(.*?)\s*```"
        matches = re.findall(pattern, query_text.strip(), re.DOTALL)
        return matches if matches else None

In [37]:
extract_query(query_text)[0]

'SELECT \n    jumia.productName,\n    jumia.jumia_price,\n    jumia.jumia_discount,\n    phoneplacekenya.productName,\n    phoneplacekenya.price,\n    phoneplacekenya.discount\nFROM\n    jumia\nJOIN\n    phoneplacekenya ON jumia.productName = phoneplacekenya.productName\nWHERE\n    jumia.jumia_price < 20000 AND phoneplacekenya.price < 20000\nORDER BY\n    (jumia.jumia_price * (\n        1 - (jumia.jumia_discount / 100)\n    )) ASC,\n    (phoneplacekenya.price * (\n        1 - (phoneplacekenya.discount / 100)\n    )) ASC;'

In [38]:
queries = extract_query(query_text)

if not queries:
    raise ValueError("No valid SQL queries could be extracted from the LLM output.")

    
print(f"Extracted {len(queries)} SQL queries: {queries}")

sql_execution_results = []  # To store results of each query
        
# Step 4: Execute each query and collect results
    
for query in queries:
        try:
            # query=query.replace('\n', ' ')
            
            print(f"Executing query: {query}")
            response = connection.execute(text(query))  # Execute the query
            result = response.fetchall()  # Fetch the results
            sql_execution_results.append(result)  # Store the result
                    
        except Exception as e:
                # try_to_close_connection()
                print(f"Error executing query: {e}")
                sql_execution_results.append(None)  # Append None if an error occurs
                
        
print(f"Database response: {sql_execution_results}")

Extracted 1 SQL queries: ['SELECT \n    jumia.productName,\n    jumia.jumia_price,\n    jumia.jumia_discount,\n    phoneplacekenya.productName,\n    phoneplacekenya.price,\n    phoneplacekenya.discount\nFROM\n    jumia\nJOIN\n    phoneplacekenya ON jumia.productName = phoneplacekenya.productName\nWHERE\n    jumia.jumia_price < 20000 AND phoneplacekenya.price < 20000\nORDER BY\n    (jumia.jumia_price * (\n        1 - (jumia.jumia_discount / 100)\n    )) ASC,\n    (phoneplacekenya.price * (\n        1 - (phoneplacekenya.discount / 100)\n    )) ASC;']
Executing query: SELECT 
    jumia.productName,
    jumia.jumia_price,
    jumia.jumia_discount,
    phoneplacekenya.productName,
    phoneplacekenya.price,
    phoneplacekenya.discount
FROM
    jumia
JOIN
    phoneplacekenya ON jumia.productName = phoneplacekenya.productName
WHERE
    jumia.jumia_price < 20000 AND phoneplacekenya.price < 20000
ORDER BY
    (jumia.jumia_price * (
        1 - (jumia.jumia_discount / 100)
    )) ASC,
    (phon